# 1. Introduction and Regulatory Background

This project presents a Monte Carlo simulation framework for estimating counterparty credit exposure in a simplified derivatives portfolio. Specifically, we simulate the exposure profiles of two uncollateralized instruments—an Interest Rate Swap (IRS) and a Foreign Exchange Forward (FX Forward)—to calculate Expected Exposure (EE), Exposure at Default (EAD), and Risk-Weighted Assets (RWA) under regulatory assumptions.

## 1.1 Background: Counterparty Credit Risk (CCR)

Counterparty Credit Risk (CCR) refers to the risk that the counterparty to a financial contract may default before the final settlement of cash flows. Unlike traditional credit risk, CCR is **market-dependent**, since the exposure changes over time with the market value of the trade.

In the context of Basel III, banks are required to calculate and report several measures of Counterparty Credit Risk (CCR), including:

- **Expected Exposure (EE)**: The average future positive exposure at a specific time horizon, computed across simulated market paths. It serves as the foundational input for regulatory capital calculations.

- **Potential Future Exposure (PFE)**: A specified quantile (e.g., 95% or 97.5%) of the exposure distribution at each time point, reflecting the level of exposure that is only exceeded in a small percentage of extreme scenarios. PFE is often used to set internal credit limits and monitor tail risk.

- **Exposure at Default (EAD)**: An estimate of the exposure at the time of counterparty default. Under the Internal Model Method (IMM), EAD is typically computed as a discounted sum of EE values, multiplied by a regulatory factor (e.g., α = 1.4).

- **Risk-Weighted Assets (RWA)**: The regulatory capital base for CCR, calculated by applying risk weights to EAD. RWA feeds directly into minimum capital requirement calculations under Basel III.


## 1.2 Simulation Motivation

Unlike static metrics such as current MtM (mark-to-market), the future exposure of derivatives depends on simulated market paths (e.g., interest rates, exchange rates). Monte Carlo simulation is therefore a widely accepted approach for modeling EE and EAD under the **Internal Model Method (IMM)** framework.

The simulation aligns with practices outlined in:


- [Basel III Counterparty Credit Risk Framework](https://www.bis.org/publ/bcbs279.htm)
- [SR 11-7: Federal Reserve Guidance on Model Risk Management](https://www.federalreserve.gov/supervisionreg/srletters/sr1107.htm)

## 1.3 Project Objectives

The objectives of this project are:
1. **Simulate market paths** for interest rates and FX rates using simplified stochastic processes.
2. **Construct a two-product derivatives portfolio**: an IRS and an FX Forward.
3. **Calculate MtM and Exposure** along each simulated path.
4. **Aggregate across paths** to compute EE, PFE (optional), and EAD.
5. **Estimate regulatory capital** using simplified RWA formulas.

The framework is modular and extensible: additional instruments, collateral features, or risk factors may be added in future iterations to build a multi-product portfolio-level CCR engine.



# 2. Derivatives Portfolio and Assumptions

We construct a simplified derivatives portfolio consisting of two uncollateralized instruments: an Interest Rate Swap (IRS) and a Foreign Exchange Forward (FX Forward). Both trades are assumed to be entered at-the-money (i.e., initial MtM = 0) and evaluated under simulated market conditions.

## 2.1 Portfolio Instruments

### 2.1.1 Interest Rate Swap (IRS)

| Parameter              | Value                         |
|------------------------|-------------------------------|
| Notional               | USD 10,000,000                |
| Contract Type          | Pay Fixed / Receive Floating  |
| Fixed Rate             | 3.00% per annum               |
| Floating Rate Index    | 6M LIBOR                      |
| Maturity               | 2 years                       |
| Payment Frequency      | Semi-annual (T = 0.5, 1.0, 1.5, 2.0) |
| Collateral             | None (Uncollateralized)       |
| Initial MtM            | 0                             |

Although LIBOR has been discontinued for new contracts, it remains widely used in financial modeling and academic settings as a proxy for floating-rate exposure. In this project, we simulate 6M LIBOR paths to represent interest rate dynamics within a stylized swap framework.

### 2.1.2 Foreign Exchange Forward (FX Forward)

| Parameter                  | Value                     |
|----------------------------|---------------------------|
| Notional                   | EUR 5,000,000             |
| Spot Rate                  | 1.10 USD/EUR              |
| Forward Rate (contracted)  | 1.12 USD/EUR              |
| Maturity                   | 1 year                    |
| Collateral                 | None (Uncollateralized)   |
| Initial MtM                | 0                         |

This FX Forward represents a forward purchase of EUR against USD. At maturity, the MtM depends on the simulated spot FX rate relative to the agreed forward rate. Exposure arises when the spot rate exceeds the contracted rate.

### 2.1.3 Collateral Assumption

All instruments are assumed to be **uncollateralized**, meaning there is no exchange of variation margin to offset daily mark-to-market movements. As a result, any positive MtM directly translates into full counterparty credit exposure. In contrast, collateralized trades governed by Credit Support Annexes (CSAs) would reduce or eliminate exposure through margining. The uncollateralized setup reflects a worst-case scenario and simplifies the simulation framework.

---

## 2.2 Risk Factors and Simulation Horizon

To value the instruments and compute counterparty exposure, we simulate the following market risk factors:

| Risk Factor           | Description              | Simulation Horizon         |
|------------------------|---------------------------|----------------------------|
| Interest Rate          | 6M LIBOR rate             | 24 monthly steps (2 years) |
| FX Rate (USD/EUR)      | Spot exchange rate        | 12 monthly steps (1 year)  |

Both risk factors are simulated under a risk-neutral measure using geometric Brownian motion (GBM). For each factor, 1,000 Monte Carlo paths are generated to approximate the distribution of future exposure values.

---

## 2.3 Discounting and Regulatory Parameters

| Parameter                  | Assumption                                                   |
|----------------------------|--------------------------------------------------------------|
| Discounting Factor         | Derived from flat 3% risk-free rate                          |
| Regulatory Multiplier (`alpha`)  | `alpha = 1.4`, per Basel III IMM guidelines ([BCBS 279](https://www.bis.org/publ/bcbs279.htm)) |
| Risk Weight (RW)           | 50%, typical for interbank counterparties                    |

All exposure-related quantities (e.g., MtM, EE, EAD) are discounted using the formula:

    DF(t) = 1 / (1 + r)^t, where r = 3%

This aligns with simplified IMM-style frameworks, where discounted expected exposures form the basis for capital estimation.


